# Neubaukompass DB Migration

## 1) Rationale

The structure of the presented projects is on a different hierarchical level than the separate units we have presented so far.
I decide to create a new database model on project level. Since the API is still in Alpha, and open wide, we can understand this as a playground for "what works and what does not".

It is implied that the API calls have to be different. I suggest we pull the project scope rather than single flats for the entire portfolio.

## 2) Structure of dataframe, vet cleanliness

In [78]:
import pandas as pd
import numpy as np

In [79]:
df = pd.read_csv('neubaukompass_20181129.csv', index_col = [0])

In [80]:
df.shape

(719, 14)

It is suggested to keep the following elements:
1. address
2. city - **default sort(by = 'city')**
3. developer
4. image_location
5. name
6. subtitle? - **we ought to extract the number of flats & types, so that is a to do item**
7. min_price - **it should be better to take them as thousands instead. Saves characters on Facebook quick replies**
8. max_price
9. min_size
10. max_size
11. completion_year
12. custom_fields - **i have added this one, exceptionally as a JSON field**

It is a learning that the unit model has been too convoluted. It may be nice to have all the information thrown in as many bins as possible, but on the project level, less is more. Since this will be shown in Facebook with a card, how much information can there be possibly needed?


**In next scraper/wrangler verions, the "no_information" string has to be replaced by np.nan to avoid conflicting dtypes.
Done below manually for quick implementation.**

In [103]:
df = df.apply(lambda x: x.replace('no_information', np.nan))
df = df.apply(lambda x: x.replace('auf Anfrage', np.nan))

## 3) Model

I choose to use text fields for all these entries except the min_price since number filtered queries will be made here.
Being aware that this may result in expensive queries and mash-ups of datatypes, we may switch types later on when transferring to a stable release.
Reason is that potentially others will enter information via forms, and any analysis of figures that will be done can perform a conversion when or after querying the database.

Furthermore, I chose not to use any calculated fields with property decorators. That messes things up and won't help with DB manipulation as per experience. 
We keep it static over there and in case we need to calculate and manipulate, we do that in whatever tool fancied after ETL.

### Model: Naming convention & access

As usual, the server is spun up at Heroku with an always-alive Dyno (since FB is quite impatient, we cannot wait for an app to wake up and send a response).
We use mighty-temple for now.

**Workspace:** python-getting-started (heroku template)

**App Name:** hello

**Model Name:** project

**Url Pattern:** r'^projects/$

**_CAREFUL:_**: App URLs are redirected entirely to hello.urls for now, except path 'admin', for obvious reasons

**Requirements & Dependencies**: Nothing new

### Model: Initial Commit

Model "Project" has been committed with adjacent views and callable API (GET, POST) to mighty temple.

Migration name was .0046_project, status **OK** raised.

## 4) Filling Migration with initial data

### Testing positive API response

In [3]:
import requests

In [14]:
res = requests.get('https://mighty-temple-99148.herokuapp.com/projects/')

In [15]:
res

<Response [200]>

Response was positive for GET. Moving on to post JSONified information from Neubaukompass.

## POST request with project data to our db via our API

In [119]:
from IPython.display import clear_output
for index, values in df.iterrows():
    clear_output(wait=True)
    progress = index / df.shape[0]
    print(f'progress: {progress}% done')
    print(f'row:{index}')
    query = values.to_dict()
    print(query)
    requests.post(url = 'https://mighty-temple-99148.herokuapp.com/projects/', data = query)

progress: 0.9986091794158554% done
row:718
{'address': '71154 Nufringen,                 ', 'city': 'stuttgart', 'completion': 'vsl. 2019', 'developer': 'GA BauArt', 'image_location': nan, 'name': 'Eigentumswohnungen am Naturpark Schönbuch', 'price': nan, 'size': nan, 'subtitle': 'Neubau von 4 Eigentumswohnungen', 'min_price': nan, 'max_price': nan, 'min_size': nan, 'max_size': nan, 'completion_year': '2019'}


All done successfully. Response times acceptable.

## Testing query

In [120]:
res = requests.get('https://mighty-temple-99148.herokuapp.com/projects/')

In [121]:
import json

In [122]:
res_df = pd.DataFrame(json.loads(res.content),)

res.content revealed initial empty entries from configuration and testing, those have been removed via psql. Hence:

In [131]:
len(res_df['min_price'].values)

470

It is evident that for IntegerFields, nan cannot be accepted. It won't help our projects anyway, since we take budget into consideration.

## Testing filter

In [141]:
requests.get('https://mighty-temple-99148.herokuapp.com/projects/?min_price=&min_price_filter=120000').content

b'[{"address":"Eichhorster Stra\xc3\x9fe 14, 12689 Berlin, Marzahn,","city":"berlin","developer":"CESA Unternehmensgruppe","image_location":"https://neubaukompass.de//bilder/objekt/13856/4-mittel.jpg","name":"Eichhorst14","subtitle":"Neubau von Eigentumswohnungen","min_price":106795,"max_price":"567300","min_size":"31.0","max_size":"122.0","completion_year":"2020","custom_fields":null},{"address":"Romain-Rolland-Stra\xc3\x9fe,  Berlin, Heinersdorf,","city":"berlin","developer":"BSK Immobilien","image_location":"https://neubaukompass.de//bilder/objekt/15964/159155-mittel.jpg","name":"Romaine Berlin-Pankow","subtitle":"Neubau von 69 Mikroapartments","min_price":100845,"max_price":"232926","min_size":"21.0","max_size":"52.0","completion_year":"2018","custom_fields":null},{"address":"Pappelallee 14-17, 14469 Potsdam, Bornstedt,","city":"berlin","developer":"PROJECT Immobilien","image_location":"https://neubaukompass.de//bilder/objekt/16245/166859-mittel.jpg","name":"Studio Living Potsdam P